# Palaeogeography

## Loading packages

In [1]:
library("divvy")
library("lwgeom")
library("h3jsr")
library("ggplot2")
library("ggpubr")
library("sf")
library("raster")
library("geojsonio")
library("sp")
library("cowplot")
library("dplyr") 

Linking to liblwgeom 3.0.0beta1 r16016, GEOS 3.8.0, PROJ 6.3.1

Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1; sf_use_s2() is TRUE


Attachement du package : ‘sf’


L'objet suivant est masqué depuis ‘package:lwgeom’:

    st_perimeter


Le chargement a nécessité le package : sp

Registered S3 method overwritten by 'geojson':
  method        from     
  print.geojson geojsonsf


Attachement du package : ‘geojsonio’


L'objet suivant est masqué depuis ‘package:base’:

    pretty



Attachement du package : ‘cowplot’


L'objet suivant est masqué depuis ‘package:ggpubr’:

    get_legend



Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:raster’:

    intersect, select, union


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union




## Create usefull functions

### Base function for jitter

In [2]:
jitter_coor <- function(x){
    return(x + runif(n = 1, -2, 2))
}

### Jitter function to avoid overlapp

In [3]:
jitter_overlapp <- function(tab){
    tab_dup <- tab[duplicated(tab[,c(8,9)]),]
    for(i in 1:nrow(tab)){
        if(tab[i, 8] %in% tab_dup[,8] & tab[i, 9] %in% tab_dup[,9]){
            tab[i, 8] <- jitter_coor(tab[i, 8])
            tab[i, 9] <- jitter_coor(tab[i, 9])
        }
    }
    return(tab)
}

### Plotting maps

In [4]:
plot_maps <-function(data_epochs,i){
    temp_CM <- paste("../../Data/PaleoDEMS/CM/", data_epochs$Age[i], "Ma_CM_v7.shp", sep = "")
    temp_CS <- paste("../../Data/PaleoDEMS/CS/", data_epochs$Age[i], "Ma_CS_v7.shp", sep = "")
    
    temp_paleo_data <- paleo_coordinate_cleaned[paleo_coordinate_cleaned$age > data_epochs$Age_min[i] & paleo_coordinate_cleaned$age < data_epochs$Age_max[i], ]
    pal_cor <- data.frame(longitude = temp_paleo_data$p_lng_PALEOMAP, latitude = temp_paleo_data$p_lat_PALEOMAP)
    
    gpm <- read_sf(temp_CM)

    gpm2 <- read_sf(temp_CS)
    
    Family <- temp_paleo_data[,1]
    
    coordinates(pal_cor) <- ~longitude + latitude
    proj4string(pal_cor) <- CRS("+proj=longlat +datum=WGS84")
    SpatialPoints(pal_cor, proj4string = CRS("ESRI:54030"))
    coords_robinson <- spTransform(pal_cor, CRS("+proj=robin +datum=WGS84"))
    coords_sf <- st_as_sf(coords_robinson)
    
    palaeo_coordinate <- gpm  %>% ggplot() +         
        geom_sf(data = bb, fill = "#D7ECF4", colour = NA) +
        geom_sf(data = gpm, fill = "#C0E6E2", colour = "black", alpha = 1) +
        geom_sf(data = gpm2, fill = "#BEB4B0", colour = "black", alpha = 1) +
        geom_sf(data = bbox) +
        geom_sf(data = coords_sf, aes(color = Family)) +
        scale_color_manual(values=color_vec_orecto) + 
        coord_sf(crs = sf::st_crs("ESRI:54030")) +
        ggtitle(data_epochs$Epoch[i]) + 
        guides(color = FALSE) + 
        theme_void() +
        theme(plot.title = element_text(hjust = 0.5))
    return(palaeo_coordinate)
}

## Setting up environement

In [5]:
sf_use_s2(FALSE)

Spherical geometry (s2) switched off



In [6]:
ras <- raster::raster(res = 5, val = 1)
ras <- rasterToPolygons(x = ras, dissolve = TRUE)
bb <- sf::st_as_sf(x = ras)
bb <- st_transform(x = bb, crs = sf::st_crs(4326))
bbox <- st_graticule(crs = st_crs("ESRI:54030"), lat = c(-89.9, 89.9), lon = c(-179.9, 179.9))

## Loading occurence data

## Paleocoordinate

In [7]:
paleo_coordinate <-read.table("../../Data/Coordinate/paleo_coordinate.tsv", sep ="\t", header = TRUE)

In [8]:
paleo_coordinate_cleaned <- jitter_overlapp(paleo_coordinate)

### Create bins

In [9]:
data_epochs <- data.frame(Epoch = c("Miocene: 20 Myrs ago", "Oligocene: 30 Myrs ago", "Eocene: 50 Myrs ago", "Paleocene: 65 Myrs ago", "Campanian: 80 Myrs ago", "Cenomanian: 95 Myrs ago", "Barremian: 125 Myrs ago", "Berriasian: 140 Myrs ago", "Kimmeridgian: 155 Myrs ago", "Callovian: 165 Myrs ago", "Pliensbachian: 185 Myrs ago"),
        Age_min = c(0, 23.031, 33.901, 56.001, 66.001, 83.601, 100.501, 125.001, 145.001, 163.501, 174.101), 
        Age_max = c(23.03, 33.9, 56, 66, 83.6, 100.5, 125, 145, 163.5, 174.1, 201.3), 
        Age = c(20, 30, 50, 65, 80, 95, 125, 140, 155, 165, 185))

### Create color palette

In [10]:
paleo_coordinate_cleaned$Family <- as.factor(paleo_coordinate_cleaned$Family)

In [11]:
color_vec_orecto <- c("Brachaeluridae" = "#8174A0", "Ginglymostomatidae"="#FFC067", "Hemiscylliidae"="#F2EEE5",
            "Mesiteiidae" = "#FCFAEE","Orectolobidae"= "#9FCDF3", "Orectolobiformes incert. fam." = "#CFCFC4", "Parascylliidae" = "#3E8999",
            "Rhincodontidae" = "#CB9DF0", "Stegostomatidae" = "#B5828C")

### Plot maps

In [12]:
fig_1 <- plot_grid(plot_maps(data_epochs,11), plot_maps(data_epochs,10), plot_maps(data_epochs,9), labels = c("A", "B", "C"), ncol = 1)

fig_2 <- plot_grid(plot_maps(data_epochs,8), plot_maps(data_epochs,7), plot_maps(data_epochs,6), labels = c("D", "E", "F"), ncol = 1)

fig_3 <- plot_grid(plot_maps(data_epochs,5), plot_maps(data_epochs,4), plot_maps(data_epochs,3), labels = c("G", "H", "I"), ncol = 1)

fig_4 <- plot_grid(plot_maps(data_epochs,2), plot_maps(data_epochs,1), plot_maps(data_epochs,1), labels = c("J", "K", "L"), ncol = 1)

Warning message:
“The `<scale>` argument of `guides()` cannot be `FALSE`. Use "none" instead as
of ggplot2 3.3.4.”


## Save maps

In [13]:
pdf(file = "Family_plot_1.pdf", width = 10, height = 10)
fig_1
dev.off()

pdf 
  2

In [14]:
pdf(file = "Family_plot_2.pdf", width = 10, height = 10)
fig_2
dev.off()

pdf 
  2

In [15]:
pdf(file = "Family_plot_3.pdf", width = 10, height = 10)
fig_3
dev.off()

pdf 
  2

In [16]:
pdf(file = "Family_plot_4.pdf", width = 10, height = 10)
fig_4
dev.off()

pdf 
  2